In [6]:
import pandas as pd

In [8]:
file_path = 'GSE140831_series_matrix.csv'

In [9]:
def extract_data_between_markers(file_path, start_marker, end_marker):
    data_between_markers = []
    capture_data = False
    with open(file_path, 'r') as file:
        for line in file:
            if start_marker in line:
                capture_data = True
                continue
            elif end_marker in line:
                capture_data = False
                continue
            if capture_data:
                data_between_markers.append(line.strip().split(','))
    return data_between_markers

# Markers to look for
start_marker = "!series_matrix_table_begin"
end_marker = "!series_matrix_table_end"

# Extract the data between the markers
extracted_data = extract_data_between_markers(file_path, start_marker, end_marker)

# Check if we have extracted any data
if extracted_data:
    # Convert the extracted data to a DataFrame
    new_dataset = pd.DataFrame(extracted_data[1:], columns=extracted_data[0])
else:
    # If no data was extracted, we return an empty DataFrame
    new_dataset = pd.DataFrame()

new_dataset.head()

,ID_REF,GSM4187187,GSM4187188,GSM4187189,GSM4187190,GSM4187191,GSM4187192,GSM4187193,GSM4187194,GSM4187195,...,GSM4188306,GSM4188307,GSM4188308,GSM4188309,GSM4188310,GSM4188311,GSM4188312,GSM4188313,GSM4188314,GSM4188315
0,0..gjSdbqnr.rUSTBI,9.906190656310688,9.015454237814241,9.464473532817756,8.860070731249948,9.367120903154984,9.810355781090696,9.383212211448615,9.278979894288938,9.870444878078992,...,8.493618056764433,8.371065845103267,8.344066139078684,8.588600015,8.689738454774684,8.472592394565616,8.384636144352886,8.501668163,8.710321064874893,8.664542432258887
1,0.a.oQZKQBR330TnQI,7.311852357359597,7.273327442773611,7.272347235419059,7.286262538973014,7.292754163439251,7.290933462303249,7.268265165159356,7.263872294122113,7.2775895196123015,...,7.476642685454135,7.469127032979658,7.511094290739461,7.443113167012164,7.509147577068849,7.528748971670505,7.471646904322741,7.493795797236957,7.487590442819114,7.4523644123699775
2,0.c4xe3uP9onqr1XFU,7.63876777,7.719423806250217,7.671300277929372,7.741190560695964,7.804022274,7.704734915612075,7.530667999915992,7.817090473814955,7.5888433477324435,...,7.613844908348298,7.563876953567448,7.556626672803354,7.6505164591612855,7.5358637683332566,7.592845910845828,7.561722028714517,7.553694912361302,7.566617210392197,7.559892935930975
3,0.D1yhJGVn_V_F6cqI,7.327134235173647,7.322934589853843,7.263823679075232,7.226814337,7.292265368190724,7.277289419542514,7.254875930658839,7.272944723,7.295090177,...,7.477756593087075,7.468587583582201,7.460508603230734,7.4888827979858155,7.4977167786359065,7.434845741105569,7.503080348,7.469751239617037,7.500043126611892,7.424349669
4,0.jegHCHIgt8Ijh90g,7.296580443611805,7.316636882294526,7.275871824665396,7.262058667349863,7.526293658269879,7.331330734596272,7.269205039540467,7.290528941376746,7.427376659185524,...,7.413099687373993,7.465294087762559,7.508042913417778,7.456684208116248,7.510767933,7.453226726020171,7.467318815497437,7.443413638474541,7.495457912304105,7.434246306165537


In [18]:
characteristics_data = pd.read_csv(file_path)

# Let's redefine the function to handle NaN or non-string inputs.
def extract_characteristic(char_str):
    if isinstance(char_str, str) and ': ' in char_str:
        return char_str.split(': ')[1]
    return None

# Read the CSV file again to extract participant characteristics
characteristics_data = pd.read_csv(file_path)

# Find the rows with participant IDs
participant_id_row = characteristics_data[characteristics_data.iloc[:, 0] == '!Sample_geo_accession']

# Find the rows with participant characteristics
characteristics_rows = characteristics_data[characteristics_data.iloc[:, 0].str.startswith('!Sample_characteristics_ch1', na=False)]

# Extract participant IDs
participant_ids = participant_id_row.iloc[0, 1:].values

# Initialize lists to store the extracted data
diagnoses = []
sexes = []
ages = []
connectivity_zscores = []
batches = []
apoe_statuses = []

# Iterate over the columns to extract characteristics
for col in characteristics_rows.columns[1:]:  # Skip the '!Series_title' column
    diagnoses.append(extract_characteristic(characteristics_rows[col].iloc[0]))
    sexes.append(extract_characteristic(characteristics_rows[col].iloc[1]))
    ages.append(extract_characteristic(characteristics_rows[col].iloc[2]))
    connectivity_zscores.append(extract_characteristic(characteristics_rows[col].iloc[3]))
    batches.append(extract_characteristic(characteristics_rows[col].iloc[4]))
    apoe_statuses.append(extract_characteristic(characteristics_rows[col].iloc[5]))

# Create the DataFrame
participant_characteristics_df = pd.DataFrame({
    'participantID': participant_ids,
    'Diagnosis': diagnoses,
    'Sex': sexes,
    'Age': ages,
    'Connectivity_Zscore': connectivity_zscores,
    'Batch': batches,
    'ApoE_Status': apoe_statuses
})

# Convert 'Age' and 'Connectivity_Zscore' to numeric types
participant_characteristics_df['Age'] = pd.to_numeric(participant_characteristics_df['Age'], errors='coerce')
participant_characteristics_df['Connectivity_Zscore'] = pd.to_numeric(participant_characteristics_df['Connectivity_Zscore'], errors='coerce')

# Display the first few rows of the DataFrame
participant_characteristics_df.head()

/tmp/ipykernel_1659/3449208992.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,

,participantID,Diagnosis,Sex,Age,Connectivity_Zscore,Batch,ApoE_Status
0,GSM4187187,Control,Male,68,-0.20,37,E3_E4
1,GSM4187188,MCI,Male,82,-0.13,21,E3_E3
2,GSM4187189,Control,Female,63,-0.32,23,E3_E3
3,GSM4187190,MCI,Male,77,0.20,37,E3_E3
4,GSM4187191,Control,Male,82,-1.40,20,E3_E3


In [19]:
main_dataset_path = 'main_dataset.csv'
new_dataset.to_csv(main_dataset_path, index=False)

# Save the participant characteristics dataset
participant_characteristics_path = 'participant_characteristics.csv'
participant_characteristics_df.to_csv(participant_characteristics_path, index=False)

# Return the paths to the saved CSV files
main_dataset_path, participant_characteristics_path

('main_dataset.csv', 'participant_characteristics.csv')